In [1]:
import urllib3
import json
import requests
import io
import pandas as pd
import zipp
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib_scalebar.scalebar import ScaleBar

# find the theme that interests me

In [160]:
def cbs_get(url, params=None):
    headers = {
        "User-Agent": "MyApp/1.0 (yodfatbenshalom@gmail.com)"  # חשוב מאוד
    }
    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    return resp.json()

In [155]:
# 1. קבלת נושאים ברמה 1
catalog_url = "https://apis.cbs.gov.il/series/catalog/level"
params = {"id": 5,"subject":2, "format": "json", "download": "false", "lang": "he"}
catalog = cbs_get(catalog_url, params)
catalog_numbers=pd.DataFrame(catalog['catalogs']['catalog'])
catalog_numbers.head()


,path,name,pathDesc
0,"[2, 1, 1, 1, 2]",סך הכל,None
1,"[2, 1, 1, 1, 378]","יהודים ואחרים (יהודים,נוצרים שאינם ערבים,ללא ס...",None
2,"[2, 1, 1, 1, 379]","אוכלוסייה ערבית (מוסלמים, דרוזים, נוצרים ערבים)",None
3,"[2, 1, 1, 1, 380]",יהודים,None
4,"[2, 1, 1, 1, 381]",לא יהודים,None


In [131]:
url='https://apis.cbs.gov.il/series/data/path?id=2,1,1,2,379&format=json&download=false'
cbs_get(url)

{'DataSet': {'Series': [{'id': 3709,
    'ffu_num': None,
    'time': {'value': 1, 'name': 'שנה'},
    'data': {'value': 1, 'name': 'נתונים מקוריים'},
    'unit': {'value': 3, 'name': 'אלפים'},
    'calc': {'value': 2, 'name': 'ממוצע'},
    'price': {'value': 0, 'name': ''},
    'path': {'level1': {'value': 2, 'name': 'אוכלוסייה'},
     'level2': {'value': 1, 'name': 'אוכלוסייה, לפי קבוצת אוכלוסייה'},
     'level3': {'value': 1,
      'name': 'אוכלוסייה קבועה (מ-2009 על בסיס מפקד 2008 )'},
     'level4': {'value': 2, 'name': 'ממוצעת'},
     'name_id': {'value': 379,
      'name': 'אוכלוסייה ערבית (מוסלמים, דרוזים, נוצרים ערבים)'}},
    'precis': 1,
    'update': '2025-10-22',
    'obs': [{'TimePeriod': '2023-01', 'Value': 2090.6},
     {'TimePeriod': '2022-01', 'Value': 2018.3},
     {'TimePeriod': '2021-01', 'Value': 1977.5},
     {'TimePeriod': '2020-01', 'Value': 1938.2},
     {'TimePeriod': '2019-01', 'Value': 1898.7},
     {'TimePeriod': '2018-01', 'Value': 1858.3},
     {'TimePer

# get data

In [157]:
# 2. נניח שבחרת קוד נושא מהרשימה — עכשיו קבלת נתונים
data_url = "https://apis.cbs.gov.il/series/data/list"
data_params = {
    "id": 3763,  # דוגמה לסדרה
    "startPeriod": "01-2000",
    "endPeriod": "12-2023",
    "format": "json",
    "lang": "he",
    "addNull": "false",
    "data_hide": "false"
}
data = cbs_get(data_url, data_params)

In [158]:
# 3. יצירת DataFrame מהנתונים
if "DataSet" in data:
    df = pd.DataFrame(data["DataSet"]['Series'])
elif "data" in data:
    df = pd.DataFrame(data["data"])
else:
    # תסתכל בשדה אחר בהתאם למבנה JSON
    print("Structure:", data.keys())
    df = pd.DataFrame(data)

In [159]:
rows = []

for _, row in df.iterrows():
    series_id = row['id']
    time_unit = row['time']['name']
    data_type = row['data']['name']
    unit = row['unit']['name']
    calc = row['calc']['name']
    series_name = row['path']['level1']['name']  # אפשר להוסיף עוד רמות אם רוצים
    
    for obs in row['obs']:
        rows.append({
            'series_id': series_id,
            'series_name': series_name,
            'time_unit': time_unit,
            'data_type': data_type,
            'unit': unit,
            'calc': calc,
            'time': obs['TimePeriod'],
            'value': obs['Value']
        })

flat_df = pd.DataFrame(rows)
flat_df

,series_id,series_name,time_unit,data_type,unit,calc,time,value
0,3763,אוכלוסייה,חודש,נתונים מקוריים,אלפים,,2023-12,9914.7
1,3763,אוכלוסייה,חודש,נתונים מקוריים,אלפים,,2023-11,9904.6
2,3763,אוכלוסייה,חודש,נתונים מקוריים,אלפים,,2023-10,9894.6
3,3763,אוכלוסייה,חודש,נתונים מקוריים,אלפים,,2023-09,9882.1
4,3763,אוכלוסייה,חודש,נתונים מקוריים,אלפים,,2023-08,9873.3
...,...,...,...,...,...,...,...,...
95,3763,אוכלוסייה,חודש,נתונים מקוריים,אלפים,,2016-01,8476.2
96,3763,אוכלוסייה,חודש,נתונים מקוריים,אלפים,,2015-12,8463.4
97,3763,אוכלוסייה,חודש,נתונים מקוריים,אלפים,,2015-11,8448.9
98,3763,אוכלוסייה,חודש,נתונים מקוריים,אלפים,,2015-10,8435.0


# get dictionaries

In [215]:
url='https://api.cbs.gov.il/Dictionary/geo/localities'
dict=cbs_get(url, params=None)
# pd.DataFrame(dict['dictionary']['data']['localities']['items'])


In [216]:
all_items = []
page = 1
page_size = 100  # fetch up to 100 items per request

while True:
    params = {"page": page, "pageSize": page_size}
    data_dict = cbs_get(url, params=params)
    items = data_dict['dictionary']['data']['localities']['items']
    all_items.extend(items)
    
    # Check if we got all items
    total = data_dict['dictionary']['data']['localities']['total']
    if len(all_items) >= total:
        break
    page += 1

HTTPError: 500 Server Error: Internal Server Error for url: https://api.cbs.gov.il/Dictionary/geo/localities?page=1&pageSize=100

In [209]:
data=dict['dictionary']['data']['localities']['items']
flattened = []
for item in data:
    loc = item['localities']
    flattened.append({
        'id': loc['ID']['id'],
        'year': loc['ID']['year'],
        'name_eng': loc['name_eng'],
        'name_heb': loc['name_heb']
    })

# Create DataFrame
df = pd.DataFrame(flattened)


In [190]:
import pandas as pd

rows = []

for _, row in df.iterrows():
    series_id = row['id']
    series_name = row['path']['level1']['name']  # example, adjust levels as needed
    
    for obs in row['obs']:
        rows.append({
            'series_id': series_id,
            'series_name': series_name,
            'time': obs['TimePeriod'],
            'value': obs['Value']
        })

flat_df = pd.DataFrame(rows)
print(flat_df.head())


KeyError: 'path'

In [191]:
rows = []

for _, row in df.iterrows():
    for obs in row['obs']:
        rows.append({
            'series_id': row['id'],
            'series_name': row['path']['level1']['name'],
            'time_unit': row['time']['name'],
            'data_type': row['data']['name'],
            'unit': row['unit']['name'],
            'calc': row['calc']['name'],
            'value': obs['Value'],
            'time': obs['TimePeriod']
        })

flat_df = pd.DataFrame(rows)
print(flat_df.head())


KeyError: 'obs'

In [69]:
flat_df

,series_id,series_name,time_unit,data_type,unit,calc,value,time
0,3709,אוכלוסייה,שנה,נתונים מקוריים,אלפים,ממוצע,2090.6,2023-01
1,3709,אוכלוסייה,שנה,נתונים מקוריים,אלפים,ממוצע,2018.3,2022-01
2,3709,אוכלוסייה,שנה,נתונים מקוריים,אלפים,ממוצע,1977.5,2021-01
3,3709,אוכלוסייה,שנה,נתונים מקוריים,אלפים,ממוצע,1938.2,2020-01
4,3709,אוכלוסייה,שנה,נתונים מקוריים,אלפים,ממוצע,1898.7,2019-01
...,...,...,...,...,...,...,...,...
95,3732,אוכלוסייה,חודש,נתונים מקוריים,אלפים,ממוצע,1923.6,2020-02
96,3732,אוכלוסייה,חודש,נתונים מקוריים,אלפים,ממוצע,1920.6,2020-01
97,3732,אוכלוסייה,חודש,נתונים מקוריים,אלפים,ממוצע,1917.4,2019-12
98,3732,אוכלוסייה,חודש,נתונים מקוריים,אלפים,ממוצע,1914.1,2019-11
